# A kódolt csengő

A barátaitokkal megbeszéltétek, hogy szerveztek egy titkos találkozót egy elhagyatott helyen. A találkozóra nem léphet be akárki, csak az aki a csöngővel a megfelelő kódot nyomja le (rövid és hosszú csöngetések megfelelő váltakozása). Viszont is az is fontos, hogy a rövid, milyen rövid és a hosszú milyen hosszú csöngetés. Csak akkor engednek be, ha ezek hossza a megfelelő intervallumba esik. 

A csöngetés hosszának méréséhez kell most egy kódot írni, és kinyomtatni azt a képernyőre.

## Mit fogsz készíteni?

Ebben a gyakorlatban alkalmazzuk a gomb kettős természetét. A gomb lenyomásával rögzítjük annak időpillanatát, majd a felengedés pillanatát is rögzítjük és a képernyőre nyomtatjuk a kettő különbségét.

## Mit tanulsz meg?

A multifunkcionális gomb elkészítésével a következőket tanulod meg:

* Hogyan tudsz egy kapcsolót ami két esemény indítására képes egy áramkörbe beiktatni
* Hogyan programozd be a Raspberry Pi GPIO kimeneteleit a **gpiozero** modullal
* Hogyan lehet függvényeket definiálni
* Hogy lehet függvényeket változókhoz/objektumokhoz rendelni
* Hogyan használjuk a *globális* változókat

## A projekt részletekre bontása

* Lenyomáskor rögzíteni az időpontot
* Elengedéskor is rögzíteni az időpontot
* Kiszámolni a kettőkülönbségét és kinyomtatni azt a képernyőre

## Áramköri elemek listája

a) [Raspberry PI](https://malnapc.hu/yis/raspberry-pi/rpi-panelek) 

b) Nyomó gomb: [itt vásárolhatsz](https://hu.farnell.com/schurter/1301-9320/switch-smd-push-12-5mm/dp/1217772?gclid=Cj0KCQjwzZj2BRDVARIsABs3l9K-ACTnuRr-dLVcDUKleNfECM3H3kWS_RfWtmXGMXVJeY9otda4dDcaAvGLEALw_wcB&gross_price=true&mckv=sSFnjERxR_dc|pcrid|434487710397|plid||kword||match||slid||product|1217772|pgrid|101346804139|ptaid|pla-389347076066|&CMP=KNC-GHU-SHOPPING-switches-relays-NEWSTRUCTURE-13-MAY-20) vagy [építhetsz]()

c) [Jumper wires female/male](https://www.ret.hu/shop/product/e-call/jumper-vezetek-szet_53-22-63) 

## A kapcsolási rajz

<img src="prog02_circuit.png" width=600 height=400 />

1) A nyomógombot lábait illesszük a breadbord mélyedéseibe. Ügyeljünk arra, hogy a nyomógomb egy (vízszintes) sorba kerülő lábai a gomb lenyomása nélkül is kapcsolatban legyenek egymással, míg a (függőlegesen) egy sorba kerülő lábai csak a lenyomás következtében lesznek összekötve. Mindezt egy műszerrel leellenprízhetjük.

2) Az egyik jumper kábelt (kék) kössük a gomb egyik lábának sorába, míg a másik végét a Raspberry Pi **GPIO 02**-es (vagy **3**-as) jelöléssel ellátott tüskéjére. Ez a tüske alapértelmezetten pull-up (magas vagy 1) módban van, azaz 3.3 V-ot mutat a földeléshez képest.

3) A másik jumper kábelt (narancssárga) kössük a gomb másik lábának sorába, míg a másik végét a Raspberry Pi **GND** (pl. **6**-os) jelöléssel ellátott tüskéjére. Ez a tüske alapértelmezetten földelés.

## A kód

Miután elkészítettük az áramkört, meg kell írnunk a kódot ami utasítja a Raspberry Pi-t, hogyan viselkedjen amikor a gombot megnyomják és hogyan amikor azt elengedik.

Először betöltjük a ```gpiozero``` modulból a ```Button``` objektumot ami lehetővé teszi, hogy a Raspi kommunikáljon a gombbal a **GPIO** tüskéken keresztül. Emellett betöltjük a **time** modult amit időmérésre fogunk majd használni.

In [ ]:
from gpiozero import Button
import time

Az objektum beolvasása után inicializálunk egy ```Button``` objektumot, amit ```button```-nak nevezünk el. Egyben megmondjuk az objektumnak, hogy a fizikai gombot a *2*-es számú **GPIO** tüskére csatlakoztattuk.

In [ ]:
button = Button(2)

Ezen a ponton definiálunk előre két változót a ```press```-t és ```release```-t és 0-át rendelünk hozzájuk. Ezek a változók fogják rögzíteni a gomb lenyomásának és felengedésének pillanatát. Ezeket a változókat globálisan is el akarjuk majd érni és módosítani, függvényekből is és azokon kívül is. 

In [ ]:
press = 0
release = 0

Ezután definiálunk két függvényt amelyből az egyik a gomb lenyomásakor hajtódik végre, míg a másik a gomb elengedésekor. A függvényt pythonban a ```def``` utasítással lehet definiálni amit a függvény neve, zárójel majd pedig kettőspont kell kövessen:

In [ ]:
def start():
    global press
    press = time.time()
    
def end():
    global release
    release = time.time()

A ```start``` függvényben kijelentjük, hogy az itt használt ```press``` nevű változó globális, tehát a függvényben történő módosítása a függvényen kívül is megmarad. A mi esetünkben először 0-át rendeltönk a ```press``` változóhoz, majd ebben a függvényben a gomb lenyomásának pillanatát: ```time.time()```. Ha a függvényen kívül hivatkoznánk a ```press``` változóra a gomb lenyomása után, akkor nem 0 hanem a gomb lenyomásának pillanata lenne az értéke. 

Ugyanezt végezzük el a ```end``` függvényben is, csak ott a ```release``` nevű globális változóhoz rendeljük a gomb felengedésének pillanatát. 

A függvények definiálása után hozzárendeljük őket a megfelelő gomb funkciókhoz:

In [ ]:
button.when_pressed = start

button.when_released = end

A ```button.when_pressed``` metódus felel annak a feladatnak a végrehajtásáért aminek a gomb lenyomásakor kell bekövetkeznie. Evvel ellentétben a ```button.when_released``` a gomb elengedésekor hajtja végre a hozzárendelt utasítást. **Nagyon fontos**, hogy az egyenlőség jobb oldalán levő függvénynevek után nincs zárójel. Evvel biztosítjuk, hogy maga a függvény kerül átadásra az új változóba (egyenlőségjel bal oldala). 

In [ ]:
while True:
    button.wait_for_press()
    button.wait_for_release()
    
    delay = release - press
    
    print(f'The button was pressed for {delay} seconds')

A kód végén elindítjuk a már megszokott végtelen ciklust: ```while True:```. Utána utasítjuk a gombot, hogy várja a jelet, amikor lenyomják ```button.wait_for_press()``` majd a jelet amikor elengedik ```button.wait_for_release()```. Ezek a függvények elindítják az időmérést és a pillanatokat a ```press``` és ```release``` változókba mentik. Innen kiszámolhatjuk mennyi ideig tartott a gomb lenyomva tartása: ```delay = release - press```, majd az értéket kinyomtathatjuk a képernyőre. Az ```f'{delay}'``` string jelölés lehetővé teszi, hogy a ```delay``` változóra a neve alapján hivatkozhassunk a stringen belül, amennyiben a változót kapcsos zárójelbe tesszük.

Ha szeretnénk megszakítani a program futását, azt a ```Ctrl+C``` billentyű kombinációval tudjuk elérni. 

## A projekt tesztelése

Miután összeszereltük az áramkört és a kódot is megírtuk, amit pl. ```button_press_length.py``` név alatt mentettünk el, megnyithatunk a Raspberry Pi operációs rendszerén egy terminált. A terminálban a ```cd 'mappa név'``` paranccsal elnavigálunk abba a mappába, ahova a ```button_press_length.py```-t elmentettük. Ott begépelve a ```python button_press_length.py``` parancsot, letesztelhetjük a programunk működését. Ha minden jól megy akkor a gomb lenyomására majd elengedésére a képernyőn megjelenik a nyomvatartás hossza másodpercekben.

Hibaüzenetek esetén ki kell deríteni mi lehetett a probléma, pl. elgépelés, egy modul hiányzik, sorok megfelelő behúzása, idézőjel lemaradása stb. A hibaüzenet legtöbbször segít abban, hogy melyik sorban találta a hibát és hogy mi volt az. Egy kis gyakorlással bele lehet jönni azok értelmezésébe, valamint interneten is rá lehet keresni a hibaüzenet jelentésére és annak lehetséges elhárítására.

## Mit lehet javítani/továbbfejleszteni?

* A nyomvatartás hosszának számítását és kiírását függvényen belülre tenni (így a while ciklus üres maradhatna).

Írd meg kommentben, hogy szerinted mivel lehetne még feldobni ezt a kis programot!

## Referencia

1) gpiozero leírás - [https://gpiozero.readthedocs.io/en/stable/recipes.html](https://gpiozero.readthedocs.io/en/stable/recipes.html)

2) time.time leírása - [https://docs.python.org/3/library/time.html](https://docs.python.org/3/library/time.html)

3) python függvények leírása - [https://www.datacamp.com/community/tutorials/functions-python-tutorial](https://www.datacamp.com/community/tutorials/functions-python-tutorial)

4) globális változók - [https://stackoverflow.com/questions/423379/using-global-variables-in-a-function](https://stackoverflow.com/questions/423379/using-global-variables-in-a-function)